In [ ]:
import cv2  
import numpy as np
import cv2
import numpy as np
from tkinter import filedialog
from tkinter import *
#trackbar callback fucntion to update HSV value
def callback(x):
    global H_low,H_high,S_low,S_high,V_low,V_high,area_l,area_h
    #assign trackbar position value to H,S,V High and low variable
    H_low = cv2.getTrackbarPos('low H','controls')
    H_high = cv2.getTrackbarPos('high H','controls')
    S_low = cv2.getTrackbarPos('low S','controls')
    S_high = cv2.getTrackbarPos('high S','controls')
    V_low = cv2.getTrackbarPos('low V','controls')
    V_high = cv2.getTrackbarPos('high V','controls')
    area_l = cv2.getTrackbarPos('area_l','controls')
    area_h = cv2.getTrackbarPos('area_h','controls')


#create a seperate window named 'controls' for trackbar
cv2.namedWindow('controls',2)
cv2.resizeWindow("controls", 550,10);


#global variable
H_low = 12
H_high = 83
S_low= 30
S_high = 232
V_low= 0
V_high = 160
area_l=100
area_h=1000

#create trackbars for high,low H,S,V 
cv2.createTrackbar('low H','controls',0,255,callback)
cv2.createTrackbar('high H','controls',179,255,callback)

cv2.createTrackbar('low S','controls',0,255,callback)
cv2.createTrackbar('high S','controls',255,255,callback)

cv2.createTrackbar('low V','controls',0,255,callback)
cv2.createTrackbar('high V','controls',255,255,callback)
cv2.createTrackbar('area_l','controls',0,5000,callback)
cv2.createTrackbar('area_h','controls',5000,5000,callback)


root=Tk()
# Leer la imagen
root.filename1 =  filedialog.askopenfilename(initialdir = r"Users\ASUS\Desktop\zerodayssoftware\deteccion de terrenos",title = "Select file",filetypes = (("image",["*.png","*.jpg","*.jpeg","*.tiff","*.txt"]),("all files","*.*")))
filenames=root.filename1
root.destroy()
while(1):
    #read source image
    img=cv2.imread(filenames)
    bgr_low=np.array([H_low, S_low, V_low], np.uint8)#np.array([76, 0, 136], np.uint8)
    bgr_high=np.array([H_high, S_high, V_high], np.uint8)#np.array([255, 255, 255], np.uint8)
    mask=cv2.inRange(img,bgr_low,bgr_high)
    img = cv2.bitwise_and(img, img, mask=mask)
    #convert sourece image to HSC color mode
#     hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     hsv_low = np.array([H_low, S_low, V_low], np.uint8)
#     hsv_high = np.array([H_high, S_high, V_high], np.uint8)
#     #making mask for hsv range
#     mask = cv2.inRange(hsv, hsv_low, hsv_high)
#     print (mask)
    #masking HSV value selected color becomes black
    res = cv2.bitwise_and(img, img, mask=mask)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cont in contours:
        areas=cv2.contourArea(cont)
        if areas>area_l and areas<area_h:
            (x,y,w,h)=cv2.boundingRect(cont)
    #         centroides.append([1-(x+.5*w)/W,(y+.5*h)/H])
    #         coordenadas.append([1-(x)/W,(y)/H,1-(x+w)/W,(y+h)/H])
            cv2.rectangle(res, (x,y), (x+w,y+h), (0,255,0),1,cv2.LINE_AA)
            cv2.circle(res,(int(x+w/2),int(y+h/2)),10,(0,255,0))
    #show image
    cv2.imshow('mask',mask)
    cv2.imshow('res',res)
    #waitfor the user to press escape and break the while loop 
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
print([H_low, S_low, V_low]+[H_high, S_high, V_high])
#destroys all window
cv2.destroyAllWindows()

## Codigo para generar shp de poligonos apartir de contornos

In [ ]:
import geopandas as gpd
import cv2
import numpy as np
from tkinter import filedialog
from tkinter import *
from shapely.geometry import Polygon
import pandas as pd
import tqdm
root=Tk()
root.filename1 =  filedialog.askopenfilename(initialdir = r"Users\ASUS\Desktop\zerodayssoftware\deteccion de terrenos",title = "Select file",filetypes = (("image",["*.png","*.jpg","*.jpeg","*.tiff","*.txt"]),("all files","*.*")),multiple=True)
filenames=root.filename1
toluca=gpd.read_file("C:/Users/ASUS/Documents/GobiernoEdomex/Agua/geoshapes/Full manzanas/Manzana_Toluca.shp")
root.destroy()
geometry=[]
for file in tqdm.tqdm(filenames):
    img = cv2.imread(file)
    cve=file.split("/")[-1].split(".")[0]
    array=[]
    for line in str(toluca[toluca["cve_cat"]==cve]["geometry"].values[0]).replace("POLYGON Z ","").replace("(","").replace(")","").split(","):
        array.append(line.split())
    shape=pd.DataFrame(array,columns=["Y","X","Z"])
    shape["X"]=shape["X"].astype("float64")
    shape["Y"]=shape["Y"].astype("float64")
    xmin=float(shape["X"].min())
    xmax=float(shape["X"].max())
    ymin=float(shape["Y"].min())
    ymax=float(shape["Y"].max())
    H,W,D=img.shape
    H_low = 0
    H_high = 179
    S_low= 0
    S_high = 255
    V_low= 93
    V_high = 255
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv_low = np.array([H_low, S_low, V_low], np.uint8)
    hsv_high = np.array([H_high, S_high, V_high], np.uint8)
    mask = cv2.inRange(hsv, hsv_low, hsv_high)
    mask=cv2.blur(mask,(5,5))
    res = cv2.bitwise_and(img, img, mask=mask)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    centroides=[]
    coordenadas=[]
    for cont in contours:
        area=cv2.contourArea(cont)
        if area>500 and area<5000:
            (x,y,w,h)=cv2.boundingRect(cont)
            centroides.append([1-(x+.5*w)/W,(y+.5*h)/H])
            coordenadas.append([1-(x)/W,(y)/H,1-(x+w)/W,(y+h)/H])
            cv2.rectangle(res, (x,y), (x+w,y+h), (0,255,0),1,cv2.LINE_AA)
            cv2.circle(res,(int(x+w/2),int(y+h/2)),10,(0,255,0))
    cv2.imshow('mask',mask)
    cv2.imshow('res',res)
    cv2.waitKey()
    cv2.destroyAllWindows()
    rectangulo=pd.DataFrame(coordenadas,columns=["y1","x1","y2","x2"])
    rectangulo["x1"]=xmax-rectangulo["x1"]*(xmax-xmin)
    rectangulo["y1"]=ymax-rectangulo["y1"]*(ymax-ymin)
    rectangulo["x2"]=xmax-rectangulo["x2"]*(xmax-xmin)
    rectangulo["y2"]=ymax-rectangulo["y2"]*(ymax-ymin)
    for i in range(len(rectangulo)):
        geometry.append(Polygon([(rectangulo["y1"][i],rectangulo["x1"][i]),(rectangulo["y2"][i],rectangulo["x1"][i]),(rectangulo["y2"][i],rectangulo["x2"][i]),(rectangulo["y1"][i],rectangulo["x2"][i])]))
gdf1=gpd.GeoDataFrame(geometry=geometry,crs=toluca.crs)
gdf1.set_crs=toluca.crs
gdf1.to_file("C:/Users/ASUS/Desktop/zerodayssoftware/deteccion de terrenos/poligonos.shp")